In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import numpy as np

from simulation import Simulation

In [5]:
# Load training data
num_sims = 10

sims = Simulation.load_simulations("..\\data\\processed_data\\normalized_training_data", 10, 64)
print(sims)
X_tra = np.array(sims)

print(X_tra)

# # Load validation data 
# num_sims = 5
# sims = []

# for i in range(num_sims):
#     sims.append(Simulation.load_simulation("..//data//normalized_training_data", i, 64))
    
# X_tra = np.array(sims)

# # Load test data
# num_sims = 10
# for i in range(num_sims):
#     sims.append(Simulation.load_simulation("..//data//normalized_training_data", i, 64))
    
# X_tra = np.array(sims)

[<simulation.Simulation object at 0x0000021FD0BF80D0>, <simulation.Simulation object at 0x0000021FDA120F40>, <simulation.Simulation object at 0x0000021FDA138160>, <simulation.Simulation object at 0x0000021FDA150AC0>, <simulation.Simulation object at 0x0000021FDA150AF0>, <simulation.Simulation object at 0x0000021FD2972EB0>, <simulation.Simulation object at 0x0000021FD2972D90>, <simulation.Simulation object at 0x0000021FC8AF22E0>, <simulation.Simulation object at 0x0000021FD1263C40>, <simulation.Simulation object at 0x0000021FDA166D90>]


In [ ]:
# %matplotlib ipympl

# Plot the scaled data
# sims[0].plot_animation()

In [ ]:
class RNN_floods(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(RNN_floods, self).__init__()
        # RNN layer
        self.rnn = nn.RNN(input_size=1, hidden_size=hidden_size, batch_first=True)

        # Output layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Reshape input to have feature dimension of 1
        x = x.unsqueeze(-1)   # Assuming input x has shape (batch, sequence)

        # RNN layer
        x, hn = self.rnn(x)   # We do not need the hidden states hn

        # Select the output of the last time step
        x = x[:, -1, :]

        # Output layer
        x = self.fc(x)

        return x

In [ ]:
# For simplicity, let's assume you have a tensor 'grids' with shape (num_samples, 90, 4, grid_size, grid_size)
# representing subsequent grids at each timestep

# grids = ...  # Load your grids

sims = [sim1, sim2]

array = np.zeros((2, 97, 4, 64, 64))
for sim in range(2):
    for t in range(97):
        array[sim, t, 0] = sims[sim].topography
        array[sim, t, 1] = sims[sim].wd[t]
        array[sim, t, 2] = sims[sim].vx[t]
        array[sim, t, 3] = sims[sim].vy[t]

grids = torch.tensor(array, dtype=torch.float32)
print(grids.shape)

In [ ]:
def evaluate_model(model, test_loader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0

    with torch.no_grad():  # No need to track gradients during evaluation
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()

    avg_test_loss = test_loss / len(test_loader)
    return avg_test_loss

def train_and_validate(model, train_loader, val_loader, criterion, optimizer, num_epochs, device, save_path):
    best_val_loss = float("inf")  # Track the best validation loss
    train_losses = []
    val_losses = []

    start_time = time.time()  # Start training time

    for epoch in range(num_epochs):
        # Training Phase
        model.train()
        total_train_loss = 0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Validation Phase
        avg_val_loss = evaluate_model(model, val_loader, criterion, device)
        val_losses.append(avg_val_loss)

        # Save Best Model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), save_path)

        if (epoch + 1) % 20 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}', f'Train Loss: {avg_train_loss:.4f}, '
                  f'Validation Loss: {avg_val_loss:.4f}', f'Best Validation Loss: {best_val_loss:.4f}')
    train_time = time.time() - start_time
    print("Training complete.")
    return train_losses, val_losses, best_val_loss, train_time

In [ ]:
train_dataset = TensorDataset(torch.tensor(X_tra, dtype=torch.float32), torch.tensor(Y_tra, dtype=torch.float32))
val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(Y_val, dtype=torch.float32))
test_dataset = TensorDataset(torch.tensor(X_tst, dtype=torch.float32), torch.tensor(Y_tst, dtype=torch.float32 ))